# Organize your machine learning experiments with ScalarStop

First organize your training, validation, and test sets with subclasses of a `DataBlob` objects.

Second, you describe the architecture of your machine learning models with subclasses of `ModelTemplate` objects.

Third, you create a `Model` subclass instance that initializes a model with a `ModelTemplate` and trains it on a `DataBlob`'s training and validation sets.

Finally, you can save the hyperparameters and training metrics from many `DataBlob`s, `ModelTemplate`s, and `Model`s into a SQLite or PostgreSQL database using the `TrainStore` client.

In [45]:
import os

import scalarstop as sp

import tensorflow as tf

### Table of Contents

#### 1. [**DataBlob**: Keeping your training dataset organized](#DataBlob:-Keeping-your-training-dataset-organized)
#### 2. [**ModelTemplate**: Parameterizing your model creation](#ModelTemplate:-Parameterizing-your-model-creation)
#### 3. [**Model**: Combine your ModelTemplate with your DataBlob](#Model:Combine-your-ModelTemplate-with-your-DataBlob)
#### 4. [**TrainStore**: Save and query your training metrics in a database](#TrainStore:-Save-and-query-your-machine-learning-metrics-in-a-database)

---

### `DataBlob`: Keeping your training dataset organized

The first step to training machine learning models with ScalarStop is to encase your dataset into a `DataBlob`.

A `DataBlob` is a set of three `tf.data.Dataset` pipelines--representing your training, validation, and test sets.

When you create a `DataBlob`, variables that affect the creation of the `tf.data.Dataset` pipeline are are stored in a nested Python dataclass named `Hyperparams`. Only store simple JSON-serializable types in the `Hyperparams` dataclass.

Creating a new `DataBlob` with `Hyperparams` looks roughly like this:

```python
from typing import List, Dict
import scalarstop as sp

class my_datablob_group_name(sp.DataBlob):
    @sp.dataclass
    class Hyperparams(sp.HyperparamsType):
        a: int
        b: str
        c: Dict[str, float]
        d = List[int]

    # ... more setup below ...
```

Then, we define three methods on our `DataBlob` subclass:
 - `set_training()`
 - `set_validation()`
 - `set_test()`

Each one of them has to create a *new* instance of a `tf.data.Dataset` pipeline with data samples and labels zipped together. Typically that looks like:

```python
# Create a tf.data.Dataset for your training samples.
samples = tf.data.Dataset.from_tensor_slices([1,2,3])

# And another tf.data.Dataset for your training labels.
labels = tf.data.Dataset.from_tensor_slices([0, 1, 0])

# And zip them together.
tf.data.Dataset.zip((samples, labels))
```

Do not apply any batching at this stage. We will do that later.

Now we'll create a `DataBlob` that contains the Fashion MNIST dataset.

In [2]:
class fashion_mnist_v1(sp.DataBlob):

    @sp.dataclass
    class Hyperparams(sp.HyperparamsType):
        num_training_samples: int
    
    def __init__(self, hyperparams):
        if hyperparams["num_training_samples"] > 50_000:
            raise ValueError("num_training_samples should be <= 50_000")
        super().__init__(hyperparams=hyperparams)
        (self._train_images, self._train_labels), (self._test_images, self._test_labels) = tf.keras.datasets.fashion_mnist.load_data()

    def set_training(self) -> tf.data.Dataset:
        samples = tf.data.Dataset.from_tensor_slices(
            self._train_images[:self.hyperparams.num_training_samples]
        )
        labels = tf.data.Dataset.from_tensor_slices(
            self._train_labels[:self.hyperparams.num_training_samples]
        )
        return tf.data.Dataset.zip((samples, labels))

    def set_validation(self) -> tf.data.Dataset:
        samples = tf.data.Dataset.from_tensor_slices(
            self._train_images[50_000:]
        )
        labels = tf.data.Dataset.from_tensor_slices(
            self._train_labels[50_000:]
        )
        return tf.data.Dataset.zip((samples, labels))

    def set_test(self) -> tf.data.Dataset:
        samples_tf = tf.data.Dataset.from_tensor_slices(
            self._test_images[50_000:]
        )
        labels_tf = tf.data.Dataset.from_tensor_slices(
            self._test_labels[50_000:]
        )
        return tf.data.Dataset.zip((samples, labels))

Here we create a `DataBlob` instance with a dictionary to set our `Hyperparams`.

The `DataBlob` name is computed by hashing your `DataBlob` subclass class name and the names and values of your `Hyperparams`.

In [3]:
datablob1 = fashion_mnist_v1(hyperparams=dict(num_training_samples=10))
datablob1.name

'fashion_mnist_v1-p166sf7xz19hg8n3mj8f93m8'

The `DataBlob` group name is by default the `DataBlob` subclass name.

In [4]:
datablob1.group_name

'fashion_mnist_v1'

In [5]:
print(datablob1.hyperparams)

fashion_mnist_v1.Hyperparams(num_training_samples=10)


Now we create another `DataBlob` instance with a different value for `Hyperparams`.

Note that it has a different automatically-generated `name`, but it'll have the same `group_name`.

In [6]:
datablob2 = fashion_mnist_v1(hyperparams=dict(num_training_samples=50))
datablob2.name, datablob2.group_name

('fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza', 'fashion_mnist_v1')

### `ModelTemplate`: Parameterizing your model creation

The `ModelTemplate` is the same concept as the `DataBlob`, but instead of three `tf.data.Dataset` s, the `ModelTemplate` creates a machine learning framework model object.

Here is an example of creating a Keras model. Building and compiling the model is parameterized by values in the `Hyperparams` dataclass.

In [7]:
class small_dense_10_way_classifier_v1(sp.ModelTemplate):

    @sp.dataclass
    class Hyperparams(sp.HyperparamsType):
        hidden_units: int
        optimizer: str = "adam"

    def new_model(self):
        model = tf.keras.Sequential(
            layers=[
                tf.keras.layers.Flatten(input_shape=(28, 28)),
                tf.keras.layers.Dense(self.hyperparams.hidden_units, activation='relu'),
                tf.keras.layers.Dense(10)
            ],
            name=self.name,
        )
        model.compile(
            optimizer=self.hyperparams.optimizer,
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'],
        )
        return model

Once again, the `ModelTemplate` has a unique name generated by hashing your subclass and the `Hyperparams`.

In [8]:
model_template = small_dense_10_way_classifier_v1(hyperparams=dict(hidden_units=3))
model_template.name

'small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs'

### `Model`:Combine your `ModelTemplate` with your `DataBlob`

`DataBlob`s and `ModelTemplate`s are not very useful until you bring them together with a `Model`.

A `Model` is an object created by pairing together a `ModelTemplate` instance and a `DataBlob` instance, for the purpose of training the machine learning model created by the `ModelTemplate` on the `DataBlob`'s training and validation sets.

Make sure to batch your `DataBlob` before using it.

In [13]:
datablob = datablob2.batch(2)

model = sp.KerasModel.from_model_template(datablob=datablob, model_template=model_template)

Once again, the `Model` has a unique name. But this time it is just a concatenation of the `DataBlob` and `ModelTemplate` names.

In [14]:
model.name

'mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza'

In [18]:
model.fit(final_epoch=2, verbose=1)

{'loss': [31.747987747192383, 2.2752344608306885],
 'accuracy': [0.05999999865889549, 0.1599999964237213],
 'val_loss': [2.746401786804199, 2.496387481689453],
 'val_accuracy': [0.10040000081062317, 0.10209999978542328]}

In ScalarStop, training a machine learning model is an idempotent operation. Instead of saying, "Train for $n$ **more** epochs," we say, "Train until the model has been trained for $n$ epochs **total**."

If we call `model.fit()` again with `final_epoch()` still at 2, we get the same metrics but no training happened.

In [19]:
model.fit(final_epoch=2, verbose=1)

{'loss': [31.747987747192383, 2.2752344608306885],
 'accuracy': [0.05999999865889549, 0.1599999964237213],
 'val_loss': [2.746401786804199, 2.496387481689453],
 'val_accuracy': [0.10040000081062317, 0.10209999978542328]}

Training ScalarStop `Model`s are idempotent because they keep track of how many epochs they have been trained for and the generated training metrics (e.g. loss, accuracy, etc.). This information is saved to the filesystem if you call `model.save()` and is loaded back from disk if you create a new `Model` object with `Model.from_filesystem_or_model_template()`.

In [32]:
os.makedirs("models_directory", exist_ok=True)

model.save("models_directory")

os.listdir("models_directory")

INFO:tensorflow:Assets written to: models_directory/mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza/assets


['mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza']

In [41]:
model2 = sp.KerasModel.from_filesystem(datablob=datablob, model_template=model_template, models_directory="models_directory")
print(model2.name)
model2.history

mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza


{'accuracy': [0.05999999865889549, 0.1599999964237213],
 'loss': [31.747987747192383, 2.2752344608306885],
 'val_accuracy': [0.10040000081062317, 0.10209999978542328],
 'val_loss': [2.746401786804199, 2.496387481689453]}

If you provide `models_directory` as an argument to `fit()`, ScalarStop will save the model to the filesystem after every epoch.

In [44]:
model2.fit(final_epoch=5, verbose=1, models_directory="models_directory")

Epoch 3/5
25/25 [==============================] - 2s 75ms/step - loss: 2.2661 - accuracy: 0.1600 - val_loss: 4.5075 - val_accuracy: 0.0799
INFO:tensorflow:Assets written to: models_directory/mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza/assets
Epoch 4/5
25/25 [==============================] - 2s 74ms/step - loss: 3.1692 - accuracy: 0.1200 - val_loss: 2.3435 - val_accuracy: 0.1027
INFO:tensorflow:Assets written to: models_directory/mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza/assets
Epoch 5/5
25/25 [==============================] - 2s 76ms/step - loss: 2.2974 - accuracy: 0.1600 - val_loss: 2.3326 - val_accuracy: 0.1028
INFO:tensorflow:Assets written to: models_directory/mt_small_dense_10_way_classifier_v1-uptyfbjofo7rqv8antxrwhjs__d_fashion_mnist_v1-3wzktz1cmz86vs1r7rbmdoza/assets


{'accuracy': [0.05999999865889549,
  0.1599999964237213,
  0.1599999964237213,
  0.11999999731779099,
  0.1599999964237213],
 'loss': [31.747987747192383,
  2.2752344608306885,
  2.266085386276245,
  3.1691651344299316,
  2.297423839569092],
 'val_accuracy': [0.10040000081062317,
  0.10209999978542328,
  0.07989999651908875,
  0.10270000249147415,
  0.10279999673366547],
 'val_loss': [2.746401786804199,
  2.496387481689453,
  4.507536888122559,
  2.34350848197937,
  2.3325793743133545]}

Once again, ScalarStop saves the model's trainining history alongside the model's weights, but this is not very convenient if you want to do large-scale analysis on the training metrics of many models at once.

A better way of storing the training metrics is to use the `TrainStore`.

### `TrainStore`: Save and query your machine learning metrics in a database

In [48]:
train_store = sp.TrainStore.from_filesystem(filename="train_store.sqlite3")
train_store

<sp.TrainStore sqlite:///train_store.sqlite3>

The `TrainStore` is also available as a Python context manager.

```python
with sp.TrainStore.from_filesystem(filename="train_store.sqlite3") as train_store:
    # use the TrainStore here

# here the TrainStore database connection is automatically closed for you.
```

We don't use it that way in this example because we want to use the TrainStore across multiple Jupyter notebook cells.

The `TrainStore` will automatically save your `DataBlob` and `ModelTemplate` name, group name, and hyperparameters to the database. And when you train a `Model`, the `TrainStore` will persist the model name and the epoch training metrics.

All of this happens automatically if you pass the `TrainStore` instance to `Model.fit()`.

In [ ]:
_ = model.fit(final_epoch=5, train_store=train_store)

Once you have some information in the `TrainStore`, you can query it for information and receive results as a Pandas `DataFrame`.

In [50]:
train_store.list_datablobs()

,name,group_name,hyperparams,last_modified
0,fashion_mnist_v1-p166sf7xz19hg8n3mj8f93m8,fashion_mnist_v1,{'num_training_samples': 10},2021-04-06 06:33:18.879441


In [51]:
train_store.list_model_templates()

,name,group_name,hyperparams,last_modified
0,small_dense_10_way_classifier_v1-uptyfbjofo7rq...,small_dense_10_way_classifier_v1,"{'hidden_units': 3, 'optimizer': 'adam'}",2021-04-06 06:33:18.881517
1,small_dense_10_way_classifier_v1-axos7t2rckz8u...,small_dense_10_way_classifier_v1,"{'hidden_units': 5, 'optimizer': 'adam'}",2021-04-06 08:14:13.075815


In [52]:
train_store.list_models()

,model_name,model_class_name,model_last_modified,datablob_name,datablob_group_name,model_template_name,model_template_group_name,dbh__num_training_samples,mth__hidden_units,mth__optimizer
0,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,KerasModel,2021-04-06 06:33:18.882921,fashion_mnist_v1-p166sf7xz19hg8n3mj8f93m8,fashion_mnist_v1,small_dense_10_way_classifier_v1-uptyfbjofo7rq...,small_dense_10_way_classifier_v1,10,3,adam
1,mt_small_dense_10_way_classifier_v1-axos7t2rck...,KerasModel,2021-04-06 08:14:13.076999,fashion_mnist_v1-p166sf7xz19hg8n3mj8f93m8,fashion_mnist_v1,small_dense_10_way_classifier_v1-axos7t2rckz8u...,small_dense_10_way_classifier_v1,10,5,adam


In [53]:
train_store.list_model_epochs()

,epoch_num,model_name,last_modified,metric__loss,metric__accuracy,metric__val_loss,metric__val_accuracy
0,3,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-07 02:12:30.542895,2.298032,0.3,8.293548,0.1004
1,4,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-06 06:33:20.717740,2.087794,0.3,2.880026,0.1070
2,5,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-06 06:33:22.474861,2.083469,0.3,2.841355,0.1069
3,6,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-06 08:14:14.833201,1.903548,0.4,5.097595,0.1029
4,7,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-06 08:14:16.569196,2.096510,0.3,4.430079,0.1022
5,8,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-06 08:14:18.277378,2.102654,0.3,4.137888,0.1022
6,9,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-06 08:14:20.003647,2.107021,0.3,4.004270,0.1022
7,10,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-06 08:14:21.713084,2.108780,0.3,3.952462,0.1022


In [54]:
model_template = small_dense_10_way_classifier_v1(hyperparams=dict(hidden_units=5))

In [55]:
model = sp.KerasModel.from_model_template(datablob=datablob, model_template=model_template)

In [56]:
_ = model.fit(final_epoch=10, train_store=train_store)

Epoch 1/10
25/25 [==============================] - 2s 76ms/step - loss: 33.6136 - accuracy: 0.1488 - val_loss: 3.0698 - val_accuracy: 0.1024
Epoch 2/10
25/25 [==============================] - 2s 80ms/step - loss: 2.3200 - accuracy: 0.2491 - val_loss: 2.4557 - val_accuracy: 0.1025
Epoch 3/10
25/25 [==============================] - 2s 79ms/step - loss: 2.2940 - accuracy: 0.2491 - val_loss: 2.4395 - val_accuracy: 0.1023
Epoch 4/10
25/25 [==============================] - 2s 76ms/step - loss: 2.2717 - accuracy: 0.2491 - val_loss: 2.4373 - val_accuracy: 0.1024
Epoch 5/10
25/25 [==============================] - 2s 72ms/step - loss: 2.2597 - accuracy: 0.2491 - val_loss: 2.4365 - val_accuracy: 0.1024
Epoch 6/10
25/25 [==============================] - 2s 70ms/step - loss: 2.2537 - accuracy: 0.2491 - val_loss: 2.4361 - val_accuracy: 0.1025
Epoch 7/10
25/25 [==============================] - 2s 71ms/step - loss: 2.2502 - accuracy: 0.2491 - val_loss: 2.4359 - val_accuracy: 0.1025
Epoch 8/10
2

In [57]:
train_store.list_model_epochs()

,epoch_num,model_name,last_modified,metric__loss,metric__accuracy,metric__val_loss,metric__val_accuracy
0,1,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-07 04:37:14.280519,18.373404,0.12,3.069818,0.1024
1,2,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-07 04:37:16.196663,2.312818,0.16,2.455680,0.1025
2,3,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-07 04:37:18.095767,2.297386,0.16,2.439520,0.1023
3,3,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-07 02:12:30.542895,2.298032,0.30,8.293548,0.1004
4,4,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-07 04:37:19.935545,2.284206,0.16,2.437326,0.1024
5,4,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-06 06:33:20.717740,2.087794,0.30,2.880026,0.1070
6,5,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-07 04:37:21.664060,2.277054,0.16,2.436525,0.1024
7,5,mt_small_dense_10_way_classifier_v1-uptyfbjofo...,2021-04-06 06:33:22.474861,2.083469,0.30,2.841355,0.1069
8,6,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-07 04:37:23.361795,2.273475,0.16,2.436139,0.1025
9,6,mt_small_dense_10_way_classifier_v1-axos7t2rck...,2021-04-06 08:14:14.833201,1.903548,0.40,5.097595,0.1029
